# Google Cloud with langchain


<h3>Tracking LangChain Executions with Aim</h3>

In this notebook we will explore three usage scenarios. To start off, we will install the necessary packages and import certain modules. Subsequently, we will configure two environment variables that can be established either within the Python script or through the terminal.

In [ ]:
%pip install -U aim
%pip install -U langchain
%pip install -U VertexAI
%pip install -U google-AppBuilder-results
%pip install -U google-cloud-secret-manager==2.16.1 google-crc32c==1.5.0
%pip install -U wikipedia

In [24]:
PROJECT_ID="ml-demo-384110"

In [ ]:
PROJECT_ID=!gcloud config get-value project

print(f"PROJECT_ID: {PROJECT_ID}")

In [25]:
import os
from datetime import datetime

from langchain.llms import VertexAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

In [26]:
from google.cloud import secretmanager
import google_crc32c


def access_secret_version(
    project_id: str, secret_id: str, version_id: str
) -> str:
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    VERBOSE = False
    
    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    # Verify payload checksum.
    crc32c = google_crc32c.Checksum()
    crc32c.update(response.payload.data)
    if response.payload.data_crc32c != int(crc32c.hexdigest(), 16):
        print("Data corruption detected.")
        return response

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    payload = response.payload.data.decode("UTF-8")
    if VERBOSE == True:
        print(f"Plaintext: {payload}")
    
    return payload

print(PROJECT_ID)

CSE_ID = access_secret_version(PROJECT_ID, "CSE_ID", "latest")
GOOGLE_API_KEY = access_secret_version(PROJECT_ID, "GOOGLE_API_KEY", "latest")


ml-demo-384110


In [27]:
import os

os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["GPLACES_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = CSE_ID

The event methods of `AimCallbackHandler` accept the LangChain module or agent as input and log at least the prompts and generated results, as well as the serialized version of the LangChain module, to the designated Aim run.

In [28]:
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1:  LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
llm = VertexAI( callbacks=callbacks)

The `flush_tracker` function is used to record LangChain assets on Aim. By default, the session is reset rather than being terminated outright.

<h3>Scenario 1</h3> In the first scenario, we will use VertexAI LLM.

In [29]:
# scenario 1 - VertexAI LLM
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
aim_callback.flush_tracker(
    langchain_asset=llm,
    experiment_name="scenario 1: Chain with multiple SubChains on multiple generations",
)

llm_result.generations

[[Generation(text='What do you call a fish with no eyes? Fsh!', generation_info=None)],
 [Generation(text="**The World is a Beautiful Place**\n\nThe world is a beautiful place,\nFull of wonder and grace.\nFrom the mountains to the sea,\nThere's beauty for all to see.\n\nThe trees stand tall and proud,\nTheir leaves a verdant shroud.\nThe flowers bloom in the sun,\nTheir petals a rainbow of fun.\n\nThe animals roam free,\nIn harmony with the sea.\nThe birds sing in the trees,\nTheir songs a symphony of peace.\n\nThe world is a beautiful place,\nFull of wonder and grace.\nLet us all take a moment", generation_info=None)],
 [Generation(text='What do you call a fish with no eyes? Fsh!', generation_info=None)],
 [Generation(text="**The World is a Beautiful Place**\n\nThe world is a beautiful place,\nFull of wonder and grace.\nFrom the mountains to the sea,\nThere's beauty for all to see.\n\nThe trees stand tall and proud,\nTheir leaves a verdant shroud.\nThe flowers bloom in the sun,\nTheir

<h3>Scenario 2</h3> The third scenario involves an agent with tools.

In [30]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

In [31]:
from langchain.tools import PythonREPLTool

p = PythonREPLTool()


In [32]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.agents.agent_types import AgentType
from langchain.llms import VertexAI

agent_executor = create_python_agent(
    llm=VertexAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={"handle_parsing_errors": True},

)
agent_executor.run(
    "What is the 10th fibonacci number?"
    )





> Entering new  chain...
I need to write a python function to calculate the fibonacci number
Action: Python_REPL
Action Input: ```
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)
```
Observation: 
Thought:I need to call the function with n = 10
Action: Python_REPL
Action Input: ```
print(fibonacci(10))
```
Observation: 55

Thought:I now know the final answer
Final Answer: 55

> Finished chain.


'55'

In [23]:
import requests
import json
import time


def genAppBuilder(query ):
    

    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()

    ACCESS_TOKEN = gcloud_token[0]
    headers = {
        'Authorization': f'Bearer {ACCESS_TOKEN}',
        'Content-Type': 'application/json; charset=UTF-8'
    }
    
    data = {"query": query, "page_size": "5", "offset": 0 }
    engine_id = ""
    project_number = "210552312048"
    version = "v1alpha"
    #print(data)
    url = f"https://discoveryengine.googleapis.com/{version}/projects/{project_number}/locations/global/collections/default_collection/dataStores/{engine_id}/servingConfigs/default_search:search" 
    response = requests.post(url, data=json.dumps(data), headers=headers)
    
    #print(response)
    json_response = json.loads(response.text)
    print(json_response)
    
    title, link, snippet, description = "", "", "", ""
    try:
      predictions = json_response['results']
            
      link = predictions[0]["document"]["derivedStructData"]["link"]
      title = predictions[0]["document"]["derivedStructData"]["title"]
      snippet = predictions[0]["document"]["derivedStructData"]["snippets"][0]['snippet']
      description = predictions[0]["document"]["derivedStructData"]['pagemap']['metatags'][0]["twitter:description"]
      image = predictions[0]["document"]["derivedStructData"]['pagemap']['cse_image'][0]["src"]



    except:
      print("An error occured calling the API.")
      print("1. Check if response was not blocked based on policy violation, check if the UI behaves the same way...")
      print("2. Try a different prompt to see if that was the problem.\n")
      print(response.text)
      # print(dir(response))

    return title, link, snippet, description, image
    


In [ ]:
title, link, snippet, description, image = genAppBuilder("fleur de mariage")
print("Title: ", title)
print("Link: ", link)
print("Description: ", description)
print("Snippet: ", snippet)
print("Image: ", image)


In [ ]:
"""Util that calls Google AppBuilder."""
from typing import Any, Dict, List, Optional

from pydantic import BaseModel, Extra, root_validator

from langchain.utils import get_from_dict_or_env


class GoogleAppBuilderAPIWrapper(BaseModel):
    """Wrapper for Google AppBuilder API.

    TODO: DOCS for using it
    1. Install google-api-python-client
    - If you don't already have a Google account, sign up.
    - If you have never created a Google APIs Console project,
    read the Managing Projects page and create a project in the Google API Console.

    2. To create an API key:
    - Navigate to the APIs & Services→Credentials panel in Cloud Console.
    TODO

    3. Setup Custom AppBuilder Engine so you can AppBuilder the entire web
    - Create a custom AppBuilder engine in this link.
    - In Sites to AppBuilder, add any valid URL (i.e. www.stackoverflow.com).

    4. Enable the Custom AppBuilder API
    - Navigate to the APIs & Services→Dashboard panel in Cloud Console.
    - Click Enable APIs and Services.
    """

    AppBuilder_engine: Any  #: :meta private:
    google_api_key: Optional[str] = None
    google_cse_id: Optional[str] = None
    k: int = 10
    siterestrict: bool = False

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid

    def _google_AppBuilder_results(self, AppBuilder_term: str, **kwargs: Any) -> List[dict]:
        cse = self.AppBuilder_engine.cse()
        if self.siterestrict:
            cse = cse.siterestrict()
        res = cse.list(q=AppBuilder_term, cx=self.google_cse_id, **kwargs).execute()
        return res.get("items", [])

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that api key and python package exists in environment."""
        google_api_key = get_from_dict_or_env(
            values, "google_api_key", "GOOGLE_API_KEY"
        )
        values["google_api_key"] = google_api_key

        try:
            from googleapiclient.discovery import build

        except ImportError:
            raise ImportError(
                "google-api-python-client is not installed. "
                "Please install it with `pip install google-api-python-client`"
            )

        service = build("customAppBuilder", "v1", developerKey=google_api_key)
        values["AppBuilder_engine"] = service

        return values

    def run(self, query: str) -> str:
        """Run query through GoogleSearch and parse result."""
        snippets = []
        results = self._google_AppBuilder_results(query, num=self.k)
        if len(results) == 0:
            return "No good Google AppBuilder Result was found"
        for result in results:
            if "snippet" in result:
                snippets.append(result["snippet"])

        return " ".join(snippets)

    def results(self, query: str, num_results: int) -> List[Dict]:
        """Run query through GoogleAppBuilder and return metadata.

        Args:
            query: The query to AppBuilder for.
            num_results: The number of results to return.

        Returns:
            A list of dictionaries with the following keys:
                snippet - The description of the result.
                title - The title of the result.
                link - The link to the result.
        """
        metadata_results = []
        results = self._google_AppBuilder_results(query, num=num_results)
        if len(results) == 0:
            return [{"Result": "No good Google AppBuilder Result was found"}]
        for result in results:
            metadata_result = {
                "title": result["title"],
                "link": result["link"],
            }
            if "snippet" in result:
                metadata_result["snippet"] = result["snippet"]
            metadata_results.append(metadata_result)

        return metadata_results


In [ ]:
"""Tool for the Google AppBuilder API."""

from typing import Optional

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools.base import BaseTool
from langchain.utilities.google_AppBuilder import GoogleSearchAPIWrapper


class GoogleGenAppBuilder(BaseTool):
    """Tool that adds the capability to query the Google AppBuilder API."""

    name = "google_genappbuilder"
    description = (
        "A wrapper around Gen App Builder. "
        "Useful for when you need to answer questions about documents / indexes indexed with your AppBuilder engine. "
        "Input should be a AppBuilder query."
    )
    api_wrapper: GoogleAppBuilderAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return self.api_wrapper.run(query)

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("GoogleAppBuilderRun does not support async")


class GoogleAppBuilderResults(BaseTool):
    """Tool that has capability to query the Google AppBuilder API and get back json."""

    name = "Google AppBuilder Results JSON"
    description = (
        "A wrapper around Google AppBuilder. "
        "Useful for when you need to answer questions about current events. "
        "Input should be a AppBuilder query. Output is a JSON array of the query results"
    )
    num_results: int = 4
    api_wrapper: GoogleAppBuilderAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool."""
        return str(self.api_wrapper.results(query, self.num_results))

    async def _arun(
        self,
        query: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("GoogleAppBuilderRun does not support async")

In [35]:
from langchain.tools import GooglePlacesTool

places = GooglePlacesTool()

ValidationError: 1 validation error for GooglePlacesAPIWrapper
__root__
  Invalid API key provided. (type=value_error)

In [34]:
# scenario 2 - Agent with Tools
tools_std = load_tools(["google-search",  "wikipedia", "llm-math"], llm=llm, callbacks=callbacks)

tools = [places]+tools_std


agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    callbacks=callbacks,
)
agent.run(
   # "What is the more expensive restaurant in the birth place of Zinedine zidane ? "
   "J’aimerais réserver un bureau jeudi prochain et une salle de réunion dans 3 jours "
)
aim_callback.flush_tracker(langchain_asset=agent, reset=False, finish=True)


NameError: name 'places' is not defined